In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data-stage2.csv')
df

,Unnamed: 0,category,amt,is_fraud,hour,trans_count_7d,trans_count_30d,time_diff
0,0,misc_net,4.97,0,1,0.0,0.0,0.000000
1,1,grocery_pos,107.23,0,1,0.0,0.0,0.000000
2,2,entertainment,220.11,0,1,0.0,0.0,0.000000
3,3,gas_transport,45.00,0,1,0.0,0.0,0.000000
4,4,misc_pos,41.96,0,1,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...
1852389,1852389,health_fitness,43.77,0,1,39.0,167.0,4.619444
1852390,1852390,kids_pets,111.84,0,1,62.0,272.0,2.706389
1852391,1852391,kids_pets,86.88,0,1,67.0,277.0,0.201111
1852392,1852392,travel,7.99,0,1,36.0,192.0,3.340278


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['is_fraud'], axis = 1),
    df[['is_fraud']],
    test_size=0.2
)

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn import set_config

preprocessor = ColumnTransformer(
    transformers=[
        ("num", MinMaxScaler(), make_column_selector(dtype_include="float64")),
        ("cat", OneHotEncoder(), make_column_selector(dtype_include="object")),
    ],
    remainder='passthrough'
)

set_config(display="diagram")

preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('num', MinMaxScaler(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x0000022EE02F57C0>),
                                ('cat', OneHotEncoder(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x0000022EE02F58E0>)])

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

clf = Pipeline(
    steps=[
        ("preprocessor", preprocessor), 
        ("classifier", RandomForestClassifier(n_jobs=-1))
    ]
)

clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', MinMaxScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000022EE02F57C0>),
                                                 ('cat', OneHotEncoder(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000022EE02F58E0>)])),
                ('classifier', RandomForestClassifier(n_jobs=-1))])

In [6]:
clf.fit(X_train, y_train.to_numpy().reshape(-1))

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', MinMaxScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000022EE02F57C0>),
                                                 ('cat', OneHotEncoder(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000022EE02F58E0>)])),
                ('classifier', RandomForestClassifier(n_jobs=-1))])

In [8]:
from sklearn.metrics import classification_report

print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    368590
           1       0.91      0.68      0.78      1889

    accuracy                           1.00    370479
   macro avg       0.95      0.84      0.89    370479
weighted avg       1.00      1.00      1.00    370479

